# Image Classification with ArcFace Loss

In this notebook we will train SOTA Computer Vision models (such as MobileNet, EfficientNet) to classify car brands. We will ArcFace Loss function for this task, which is also state-of-art.

# 1. Set up

## 1. Packages and requirements

In [ ]:
# Major builtin libraries
import os
import gc
import math
import time
import random
import typing as t
from copy import deepcopy
# from collections import defaultdict

In [ ]:
import warnings  # If you want to disable warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
# Classic packages for data manipulation and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Basic PyTorch
import torch
import torch.nn as nn
import torch.optim as optim  # Optimization algorithms and dynamic learning rate adjusting
import torch.nn.functional as F
# from torch.nn.modules.loss import _Loss  # For writing a custom Loss function
from torch.utils.data import DataLoader, Dataset  # For custom data presentation

In [ ]:
# Utils
import joblib  # Pipelining, pickling (dump/load), parallel processing
from tqdm import tqdm  # Progress bar for training process
from tempfile import TemporaryDirectory

# Classic ML tools
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold  # Cross-Validation

In [ ]:
!pip install -u scikit-learn

In [ ]:
# Data dimensionality reduction
from sklearn.manifold import TSNE  # high-dimensional data visualization
from sklearn.decomposition import PCA  # More effective dimensionality reduction method

# Clustering methods
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

In [ ]:
# ML Metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from torchmetrics.classification import MulticlassF1Score # F1 metric for multiclass

In [ ]:
# Torch Computer Vision tools for images processing
from torchvision.io import read_image
from torchvision.transforms.functional import to_pil_image, to_grayscale, to_tensor
from torchvision import models  # Pretrained models

In [ ]:
# Albumentations is an OS library for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
# Output text colorizing
from colorama import Back, Style

def print_highlighted(text: str, bgcolor=Back.YELLOW) -> None:
    """
    Function to print a text with colored background.
    """
    print(bgcolor + text + Style.RESET_ALL)

In [ ]:
# import wandb # MLOps platform to simplify and speed up the process of building ML models

In [ ]:
# wandb.login() # We log in via pop-up,

## 2. Configuration

In [ ]:
CONFIG = {
    "seed": 2306,
    "epochs": 20,
    "image_dimension": 256,  # Depends on pretrained model used
    "model_name": "efficientnet_b2",  # Pretrained model we will use
    "embedding_size": 512,  # Embedding output size
    "train_batch_size": 64,
    "val_batch_size": 128,
    "learning_rate": 1e-3,
    "min_lr": 1e-8,
    "min_loss_delta": 1e-7, # To stop training on plateau
    "weight_decay": 1e-7,
}

In [ ]:
class Config:
    """
    Class to simplify access to Config: not dict but object with attributes
    """
    def __init__(self, config_dict):
        for key, value in config_dict.items():
            setattr(self, key, value)

In [ ]:
config = Config(config_dict=CONFIG)
del CONFIG

In [ ]:
# Work example
config.seed

Set Seed for Reproducibility

In [ ]:
def set_seed(seed=42):
    """
    Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.
    """
    np.random.seed(seed)
    torch.manual_seed(seed)

    # When running on the CuDNN backend, two further options must be set
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = False  # When False, this option makes CUDA reproducible, BUT the performance might suffer

    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(seed=config.seed)

# 2. Data

## 1. Location

In [ ]:
config.repo = 'car-brands/'  # dataset name on Kaggle

config.root = '/kaggle/input/' + config.repo

config.data = config.root + 'dataset/'

config.train_images_path  = config.data + 'images/train'
config.test_images_path = config.data + 'images/test'

config.train_labels  = config.data + 'train_labels.csv'
config.test_labels = config.data + 'test_labels.csv'

# config.model_path = '/kaggle/input/efficient-model/EfficientNet_b2.pth'
config.model_path = '/kaggle/input/efficient-model/ArcFace_efficientnet_b2.pth'

## 2. Custom Data representation

In [ ]:
def get_file_path_by_id(file_id, dir=config.root):
    return os.path.join(dir, str(file_id) + ".jpg")

In [ ]:
def apply_label_encoding(data: t.Union[pd.DataFrame, np.array],
                         encoder_name: os.path,
                         column='label', action='encode',
                         ):
    """
    One Hot encoding. We apply encoding by replacing the label column in dataframe.
    As for decoding data back, we work with vector-array (as it's most likely to
    be a prediction result)
    """
    encoder = LabelEncoder()
    if action == 'encode':
        # We transform dataframe here. Nothing returns
        data[column] = encoder.fit_transform(data[column])
        with open(f"{encoder_name}_LEncoder.pkl", "wb") as fp:
            joblib.dump(encoder, fp)
    elif action == 'decode':
        # We pass vector here. Result is a vector
        with open(f"{encoder_name}_LEncoder.pkl", "rb") as fp:
            encoder: LabelEncoder = joblib.load(fp)
        if type(data) == torch.Tensor:
            data = data.cpu().numpy()
        return encoder.inverse_transform(data)

In [ ]:
class CustomImagesDataset(Dataset):
    """
    """
    def __init__(self, data: pd.DataFrame=None, images_path: os.path=None, labels_path:os.path=None, transform_images: A.Compose=None, encoder_name: str=None):
        """

        """
        super().__init__()
        assert (data is not None) or (labels_path is not None and images_path is not None)

        if data is None:
            data = pd.read_csv(labels_path)
            data['file_path'] = data['id'].apply(get_file_path_by_id, dir=images_path)

        self.images_paths = data['file_path'].values
        self.encoder_name = encoder_name if encoder_name else self.__hash__()  # We use hash as a unique name
        print_highlighted(f"Label Encoder saved with id `{self.encoder_name}`")
        apply_label_encoding(data=data, action='encode', encoder_name=self.encoder_name)
        self.labels = data['label'].values
#         self.indexes = data['id'].values
        self.transform_images = transform_images
        self.__set_dataset_len()

    def __set_dataset_len(self):
        self.length = self.labels.shape[0] # Number of rows

    def __len__(self):
        """
        We calculate the len in another function, so that we are able to slice.
        """
        return self.length

    def decode_labels(self, labels_vector):
        return apply_label_encoding(data=labels_vector, action='decode', encoder_name=self.encoder_name)

    def __getitem__(self, index) -> tuple[torch.Tensor, int]:
        """ Function to return item by indexing the dataset """

        if not isinstance(index, int) and isinstance(index, slice):
            # It's not an index, but slice.
            # We will return the part of data by making a copy of the dataset
            index: slice
            self = deepcopy(self)
            self.length = index.stop  # Cut the length of dataset.
            self.labels = self.labels[:self.length]
            return self
        assert self.__len__() >= index

        image = to_pil_image(read_image(self.images_paths[index]))
        if self.transform_images:
            # Albumentations requires us to convert image to Numpy Array
            image = self.transform_images(image=np.array(image))['image']

        label = self.labels[index]
        return image, label


    @property
    def classes_(self):
        return self.decode_labels(np.unique(self.labels))


## 3. Augmentations

We use Albumantations lib

In [ ]:
data_transforms = {
    "train": A.Compose([
        A.Resize(config.image_dimension, config.image_dimension),
#         A.RandomCrop(width=224, height=224, p=1),
        A.OneOf([
            A.Resize(224, 224, p=0.5),
            A.RandomCrop(width=224, height=224, p=1.0),
        ], p=1.0),
        A.ShiftScaleRotate(
            shift_limit=0.1,
            scale_limit=0.1,
            rotate_limit=30,
            p=0.5),

        A.HueSaturationValue(
                hue_shift_limit=30,
                sat_shift_limit=30,
                val_shift_limit=30,
                p=0.5
            ),
        A.RandomBrightnessContrast(
                brightness_limit=(-0.01,0.01),
                contrast_limit=(-0.01, 0.01),
                p=0.5
            ),
        A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
        ToTensorV2() # Advanced analogue of T.ToTensor
    ], p=1.),

    "val": A.Compose([
        A.Resize(config.image_dimension, config.image_dimension),
        A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
        ToTensorV2()], p=1.)
}

## 4. Load the data and have a look

In [ ]:
train_dataset = CustomImagesDataset(labels_path=config.train_labels, images_path=config.train_images_path, transform_images=data_transforms['train'], encoder_name='train')

In [ ]:
test_dataset = CustomImagesDataset(
    labels_path=config.test_labels,
    images_path=config.test_images_path,
    transform_images=data_transforms['val'],
    encoder_name='test',
)

In [ ]:
to_pil_image(train_dataset[60][0])

In [ ]:
config.num_of_classes = len(train_dataset.classes_)
config.num_of_classes

Below we ensure that the test dataset doesn't contain unknown data

In [ ]:
assert not bool(set(test_dataset.classes_) - set(train_dataset.classes_))

In [ ]:
print(*train_dataset.classes_[:20], sep=';  ', end='...')

# 3. Model

ArcFace loss function is used on the embeddings, not on the final predictions

## 1. Getting the pretrained model

In [ ]:
def get_input_feature_size(classifier: nn.Sequential) -> int:
    for module in classifier.modules():
        if isinstance(module, nn.Linear):
            return module.in_features

In [ ]:
def get_model(model_name='resnet18', from_path=None, pretrained=False, freeze_layers=0.0, dropout=0.1, get_embeddings=False, transform_classifier=False) -> torch.nn.Module:
    if from_path:
        try:
            model = torch.load(from_path, map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
        except FileNotFoundError:
            raise
    elif model_name:
        model = getattr(models, model_name) # We use builtin function
        model = model(
        weights=('DEFAULT' if pretrained else None)
        )

    if bool(freeze_layers):
        params = list(model.parameters())
        for param in params[:int(len(params)*freeze_layers)]:
            # Freeze some layers
            param.requires_grad = False

    if get_embeddings:
        # That means remove classifier (last layer):
        model.classifier = nn.Sequential(
            nn.Dropout(p=0.2, inplace=True),
            nn.Linear(in_features=get_input_feature_size(model.fc),
                      out_features=config.embedding_size, bias=False
                      ),

        )
    elif transform_classifier:
        try:
            output_layer = model.classifier
        except AttributeError:
            output_layer = model.fc
            
        model.classifier = nn.Sequential(
            nn.Linear(in_features=get_input_feature_size(output_layer), out_features=config.num_of_classes, bias=True),
            nn.Dropout(p=dropout, inplace=True),
#             nn.Softmax(dim=-1)
         )
    return model

We can fine-tune the model or train from scratch

In [ ]:
# model = get_model(
#     model_name=config.model_name,
#     pretrained=True,
#     freeze_layers=0.0,
#     get_embeddings=True
# )

Or fine-tine the model that we've trained before

In [ ]:
model = get_model(
    # We don't apply transformations when we use saved model
    from_path=config.model_path,
    pretrained=True,
    dropout=0.2,
)

It is a model that doesn't have classifier - only one linear layer that transforms embeddings to desired size

In [ ]:
model.classifier

**Number of model parametres:**

In [ ]:
print_highlighted(str(sum([np.prod(p.size()) for p in model.parameters()])))

# 4. Various

## 1. Quality metrics

We will use accuracy and F1 score for out classification task

In [ ]:
f1_score = MulticlassF1Score(num_classes=config.num_of_classes, average="micro")

## 2. Device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
f1_score = f1_score.to(device)
torch.cuda.empty_cache()

## 3. Data processing format

In [ ]:
config.train_batch_size, config.val_batch_size = 96, 256

In [ ]:
train_dataloader = DataLoader(train_dataset,
                              batch_size=config.train_batch_size,
                              shuffle=True,
                              num_workers=os.cpu_count() % 4,
)

In [ ]:
test_dataloader = DataLoader(test_dataset,
                              batch_size=config.val_batch_size,
                              shuffle=False,
                              num_workers=os.cpu_count() % 4,
)

# 5. Initialize the train process

## 1. Functions to plot the result

In [ ]:
def plot_two_curves(
        title: str,
        train_curve: list[float],
        val_curve: list[float],

):
    plt.figure(figsize=(16, 8))
    plt.title(title)
    plt.plot(train_curve, label=f'Train {title}')
    plt.plot(val_curve, label=f'Valid {title}')
    plt.legend()
    plt.grid()
    plt.show()
    #plt.savefig(title)

In [ ]:
def plot_stats(
        train_loss: list[float],
        valid_loss: list[float],
        train_accuracy: list[float]=None,
        valid_accuracy: list[float]=None,
        train_f1_score: list[float]=None,
        valid_f1_score: list[float]=None,
        title: str = None
):
    plot_two_curves(
        'Loss',
        train_loss,
        valid_loss
    )

    if train_accuracy and valid_accuracy:
        plot_two_curves(
            'accuracy',
            train_accuracy,
            valid_accuracy
        )

    if train_f1_score and valid_f1_score:
        plot_two_curves(
            'F1 score',
            train_f1_score,
            valid_f1_score
        )


## 2. Training and evaluation functions

In [ ]:
def train_one_epoch(model, dataloader, optimizer, criterion, device, epoch: str, margin=None):
    model.train()

    train_loss = 0.0
    train_correct = 0.0
    train_f1 = 0.0
    n_batches = 0
    data_size = 0

    with tqdm(dataloader, desc=f'Epoch: {epoch}', unit="batch") as tepoch:
        for images, labels in tepoch:
            n_batches += 1
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            embeddings = model.features(images)
            embeddings = torch.squeeze(model.avgpool(embeddings), dim=(2, 3))  # From (B, 1408, 1, 1) -> (B, 1408)
            embeddings = F.normalize(embeddings, dim=1)
            
            weights = model.classifier[0].weight
            weights = F.normalize(weights, dim=1)
            
#             # Compute the cosine similarity between embeddings and weights
            cosine = F.linear(embeddings, weights)
            del embeddings
            del weights
        
            logits = margin(cosine, labels)
            del cosine
            
#             logits = model(images)

            loss = criterion(logits, labels)
            
            
#             del weights
            
#             loss = F.cross_entropy(probabilities, labels)
            loss.backward()

            optimizer.step()
            
            gc.collect()

            loss = loss.item()  # item() extracts scalar
            train_loss += loss

            _, predicted = torch.max(logits, 1)
            del logits

            train_correct += (predicted == labels).sum().item()
            data_size += labels.size(0)
            train_accuracy = train_correct / data_size

            f1 = f1_score(predicted, labels).item()
            train_f1 += f1
            tepoch.set_postfix(loss=loss, acc=train_accuracy)

    train_loss /= n_batches
    train_f1 /= n_batches
    gc.collect()
    torch.cuda.empty_cache()

    return train_loss, train_accuracy, train_f1

In [ ]:
@torch.inference_mode()
def validate_one_epoch(model, dataloader, criterion, device):
    model.eval()

    val_loss = 0.0
    val_correct = 0.0
    val_f1_score = 0.0
    data_size = 0
    n_batches = 0

    with tqdm(dataloader, desc="Validation",unit="batch") as tepoch:
        for images, labels in tepoch:
            n_batches += 1
            images = images.to(device)
            labels = labels.to(device)
            
            
#             embeddings = model(images)
#             probabilities = criterion(embeddings, labels)
#             probabilities = criterion.predict(embeddings)
#             loss = F.cross_entropy(probabilities, labels).item()

            output = model(images)

            loss = criterion(output, labels).item()
    
            
            val_loss += loss

            _, predicted = torch.max(output, 1)

            correct = (predicted == labels).sum().item()
            val_correct += correct
            data_size += labels.size(0)
            val_accuracy = val_correct / data_size

            f1 = f1_score(predicted, labels).item()
            val_f1_score += f1
            tepoch.set_postfix(loss=loss, acc=val_accuracy)

    val_loss /= n_batches
    val_f1_score /= n_batches
    gc.collect()
    torch.cuda.empty_cache()

    return val_loss, val_accuracy, val_f1_score

## 3. Training cycle

In [ ]:
def train_model(model, train_loader, val_loader,
                criterion=None, optimizer=None, 
                margin=lambda x, y: x, 
                num_epochs=5, scheduler=None, 
                device=torch.device("cpu"),
                save_model_to=None, save_margin_parameters_to=None
       ) -> nn.Module:
    since = time.time()

    train_loss_history, valid_loss_history = [], []
    train_acc_history, valid_acc_history = [], []
    train_f1_history, valid_f1_history = [], []


    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        for epoch in range(num_epochs):
            train_loss, train_accuracy, train_f1 = train_one_epoch(model, train_loader, optimizer, criterion, device, epoch=f'{epoch+1}/{num_epochs}', margin=margin)
            val_loss, val_accuracy, val_f1 = validate_one_epoch(model, val_loader, criterion, device,)


            print_highlighted(
                f"Loss: {val_loss:.4f} - Accuracy: {val_accuracy:.4f} - F1: {val_f1:.4f} - LR: {get_lr(optimizer)}"
            )

            if not valid_loss_history or val_loss < valid_loss_history[-1]:
                # If it's either first step (history is empty) or our weights are getting better
                print_highlighted("New weights were applied.\n", bgcolor=Back.CYAN)
                torch.save(model.state_dict(), best_model_params_path)

            scheduler.step(val_loss)

            train_loss_history.append(train_loss)
            valid_loss_history.append(val_loss)
            train_acc_history.append(train_accuracy)
            valid_acc_history.append(val_accuracy)
            train_f1_history.append(train_f1)
            valid_f1_history.append(val_f1)

            del train_loss
            del val_loss
            gc.collect()
            torch.cuda.empty_cache()

        time_elapsed = time.time() - since
        print_highlighted(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))

        if save_model_to:
            torch.save(model, save_model_to)
            print(f"Model was saved to: {save_model_to}")
        if save_margin_parameters_to:
            torch.save(criterion.state_dict(), save_margin_parameters_to)
            print(f"Loss Function weights were saved to: {save_margin_parameters_to}")


        history = dict(
            train_loss=train_loss_history,
            valid_loss=valid_loss_history,
            train_accuracy=train_acc_history,
            valid_accuracy=valid_acc_history,
            train_f1_score=train_f1_history,
            valid_f1_score=valid_f1_history,
        )
    return model, history

# 6. Hyperparametres

## 1. ArcFace Loss

Custom Loss Function

$$L = - \frac{1}{N} \sum_{i=1}^{N}\; \log \frac{e^{s(\cos(\theta_{y_i} +m))}}
{e^{s\cos(\theta_{y_i} +m)} + \sum_{j=1, j\neq {y_i}}^{n} e^{s\cos(\theta_{j})}}$$

In [ ]:
class ArcFaceMargin(nn.Module):
    def __init__(self, num_classes, margin=0.5, scale=64, pretrain_without_margin=False):
        super().__init__()
        self.num_classes = num_classes
        self.margin = margin
        self.scale = scale
        print_highlighted(f"ArcFace margin. Params: s={scale}; m={margin}.")
        
    
    def forward(self, cosine: torch.Tensor, labels: torch.Tensor):
        """
        Forward function takes backbone logits (last layer before the softmax), and applies margin to them
        """
        labels = labels.to(torch.long)

        # Calculate the theta (angle) values for each class
        theta = torch.acos(torch.clip(cosine, -1 + 1e-7, 1 - 1e-7))

        # Apply the ArcFace margin and calculate logits
        
        one_hot_labels = F.one_hot(labels, self.num_classes)
        
        # Easy margin
        target_logits = torch.cos(theta + self.margin * (1 - one_hot_labels.float()))

        logits = torch.where(one_hot_labels.bool(), target_logits, cosine)
        
        logits = self.scale * logits
        
        return logits


In [ ]:
s = 2 ** 0.5 * np.log(config.num_of_classes - 1)

In [ ]:
criterion = nn.CrossEntropyLoss()

arcface_margin = ArcFaceMargin(num_classes=config.num_of_classes, margin=0.5 * (s / 64)**2, scale=s)

In [ ]:
config.learning_rate = 1e-4
config.epochs = 20
config.weight_decay = 1e-7
config.min_loss_delta = 0.01

In [ ]:
optimizer = torch.optim.Adam(
    [
    {'params': filter(lambda p: p.requires_grad, model.features.parameters())},
    {'params': model.classifier.parameters(), 'lr': 0.01},
    ],
    lr=config.learning_rate, 
    weight_decay = config.weight_decay)


In [ ]:
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5, verbose=False)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 'min', patience = 5, factor=0.5, 
    min_lr=config.min_lr , verbose=False,
    threshold = config.min_loss_delta
)

In [ ]:
def get_lr(optimizer):
    """
    For some schedulers we don't have `get_last_lr()` method.
    So, we will get last lr ourselves.
    """
    return list(
        [group['lr'] for group in optimizer.param_groups]
    )

In [ ]:
get_lr(optimizer)

# 7. Run the training process

In [ ]:
model_file_name = f'ArcFace_{config.model_name}.pth'
# arcface_parameters = 'Trained_ArcLoss_parameters.pth'

model, history = train_model(
    model=model,
    train_loader=train_dataloader,
    val_loader=test_dataloader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=config.epochs,
    device=device,
    scheduler=scheduler,
    margin=arcface_margin,
    save_model_to=model_file_name,
#     save_margin_parameters_to=arcface_parameters
)

In [ ]:
plot_stats(
    **history,
    title=f"Model {config.model_name}"
)

# 8. Result evaluation

In [ ]:
model.eval()  # Set the model to evaluation mode
predictions = []
correct = 0
total = 0

with torch.no_grad():
    with tqdm(test_dataloader, desc="Processing",unit="batch") as process:
        for images, labels in process:
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            _, predicted = torch.max(output, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            predictions.extend(predicted.cpu().numpy())

            
print_highlighted(f"Manually calculated accuracy: {correct/total}")

In [ ]:
c_report = classification_report(test_dataset.labels, np.array(predictions), output_dict=True, zero_division=0)
c_report = pd.DataFrame(c_report).transpose()
c_report

In [ ]:
c_report[c_report['precision'] > 0][:-3].mean()

In [ ]:
accuracy_score(test_dataset.labels, np.array(predictions))

To download the model, use code below:

In [ ]:
from IPython.display import FileLink
FileLink(model_file_name)